추천 시스템에서 SVD와 MF를 같은 의미로 혼동해서 사용하곤 하는데, 실제로는 다르다.
SVD, NMF 등에서는 null값을 0이나 평균으로 대체한 후 행렬을 분해한다. 이는 학습에 영향을 주어 학습된 모델이 null값을 0 혹은 평균에 근사하게 예측하도록 한다.
실제 추천시스템에서 사용하는 MF에서는 평가가 이루어진 데이터만 사용해서 학습한다.

다만 Surprise 패키지에서는 MF가 SVD라는 이름으로 구현되어있음; 혼동하지 말 것

유저/아이템 행렬을 최적화시키는 방법에는 대표적으로 SGD와 ALS가 있음

1. SGD(Stochastic Gradient Descent)
학습데이터를 샘플링해서 예측치와의 오차 제곱을 기울기 방향으로 감소시키는 방식
2. ALS(Alternating Least Square)
사용자 행렬과 아이템 행렬을 교대로 최적화해 나감. 사용자 행렬을 최적화시킬 때는 아이템 행렬은 고정. 아이템 행렬을 최적화 시킬 때도 마찬가지. 목적함수인 실제 평가값 - 예측 평가값의 오차제곱 함수가 convex(아래로 볼록한 함수)가 아니라 최적화가 느려지는 것을 방지함.

surprise의 svd는 최적화 방식으로 SGD를 사용

In [52]:
import os
import sys

sys.path.insert(0, f'{os.environ.get("HOME")}/workspace/recommendation-study')

In [70]:
from util.data import DataLoader
from util.models import Dataset, RecommendResult
from recommend.base import BaseRecommender
from collections import defaultdict
import numpy as np
from surprise import SVD, Reader
import pandas as pd
from surprise import Dataset as SurpriseDataset
from surprise.model_selection import GridSearchCV
np.random.seed(0)

In [48]:
dataset = DataLoader().load()

In [74]:
class MFRecommender(BaseRecommender):
    def train(self, dataset: Dataset, **kwargs):
        n_factors = kwargs.get('n_factors', 5)
        minimum_num_rating = kwargs.get('minimum_num_rating', 100)
        use_bias = kwargs.get('use_bias', False)
        learning_rate = kwargs.get('learning_rate', 0.005)
        n_epochs = kwargs.get('n_epochs', 50)

        self.train_data = dataset.train.groupby('movie_id').filter(lambda x: len(x['movie_id']) >= minimum_num_rating)
        reader = Reader(rating_scale=(0.5, 5))
        surprise_train_data = SurpriseDataset.load_from_df(
            self.train_data[['user_id', 'movie_id', 'rating']], reader
        ).build_full_trainset()

        mf = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=learning_rate, biased=use_bias)
        mf.fit(surprise_train_data)

        data_test = surprise_train_data.build_anti_testset(None)
        self.predictions = mf.test(data_test)

    def grid_search(self, dataset: Dataset, param_grid: dict, minimum_num_rating: int = 100):
        train_data = dataset.train.groupby('movie_id').filter(lambda x: len(x['movie_id']) >= minimum_num_rating)
        reader = Reader(rating_scale=(0.5, 5))
        surprise_train_data = SurpriseDataset.load_from_df(
            train_data[['user_id', 'movie_id', 'rating']], reader
        )

        grid = GridSearchCV(SVD, param_grid=param_grid, measures=['rmse', 'mae'], cv=3)
        grid.fit(surprise_train_data)

        return grid

    def get_top_n(self, n: int = 10):
        top_n = defaultdict(list)
        for uid, iid, r_ui, est, _ in self.predictions:
            top_n[uid].append((iid, est))

        for uid, estimations in top_n.items():
            estimations.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = [x[0] for x in estimations[:n]]

        return top_n

    def recommend(self, dataset: Dataset, k: int, **kwargs) -> RecommendResult:
        pred_user2items = self.get_top_n(n=k)
        movie_rating_predict = dataset.test.copy()
        for uid, iid, _, est, _ in self.predictions:
            movie_rating_predict.loc[(movie_rating_predict['user_id']==uid) & (movie_rating_predict['movie_id']==iid), 'rating_pred'] = est
        
        movie_rating_predict.fillna(self.train_data.rating.mean(), inplace=True)

        return RecommendResult(movie_rating_predict.rating_pred, pred_user2items)

In [88]:
recommender = MFRecommender()
param_grid = {
    'n_epochs': [30, 50],
    'n_factors': [5, 10, 20, 50],
    'biased': [True, False],
}
grid = recommender.grid_search(dataset, param_grid=param_grid)
print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8437284503609203
{'n_epochs': 30, 'n_factors': 20, 'biased': False}


In [87]:
recommender = MFRecommender()
param_grid = {
    'n_epochs': [30, 50],
    'n_factors': [5, 10, 20, 50],
    'biased': [True, False],
}
grid = recommender.grid_search(dataset, param_grid=param_grid, minimum_num_rating=300)
print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8437584055950028
{'n_epochs': 30, 'n_factors': 5, 'biased': True}


In [91]:
recommender = MFRecommender()
param_grid = {
    'n_epochs': [30, 50],
    'n_factors': [5, 10, 20, 50],
    'biased': [True, False],
}
grid = recommender.grid_search(dataset, param_grid=param_grid, minimum_num_rating=10)
print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8612382513875646
{'n_epochs': 30, 'n_factors': 5, 'biased': False}


In [89]:
recommender = MFRecommender()
recommender.train(dataset, n_epochcs=30, n_factors=20, use_bias=False)
metrics = recommender.run_sample()
print(metrics)

rmse: 1.048, precision@K: 0.011, recall@K: 0.037


In [90]:
recommender = MFRecommender()
recommender.train(dataset, n_epochcs=30, n_factors=5, use_bias=True, minimum_num_rating=300)
metrics = recommender.run_sample()
print(metrics)

rmse: 1.137, precision@K: 0.014, recall@K: 0.047


In [93]:
recommender = MFRecommender()
recommender.train(dataset, n_epochcs=50, n_factors=5, use_bias=False, minimum_num_rating=300)
metrics = recommender.run_sample()
print(metrics)

rmse: 1.139, precision@K: 0.016, recall@K: 0.053
